### 0 - Bibliotecas

In [1]:
## Processamento de dados
import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
import random

#Bibliotecas utilizadas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

## Vusualização de dados
import matplotlib.pyplot as plt
from IPython.display import Image

### pip install --upgrade scikit-learn
%matplotlib inline

In [2]:
import warnings 
warnings.filterwarnings("ignore")

### 1 - Dado Original

In [3]:
# Leitura do dataset
df = pd.read_csv('forestfires.csv')
# Transformação da dimensão 'área' pelo seu logaritmo (depois de somar 1 para evitar zeros)
df['Logarea']=np.log10(df['area']+1)
# Codificação de rótulos para dimensões categóricas (dia e mês)
# Codificação MES
enc = LabelEncoder()
enc.fit(df['month'])
enc.classes_
df['month_encoded']=enc.transform(df['month'])
# Codificação DIA
enc.fit(df['day'])
enc.classes_
df['day_encoded']=enc.transform(df['day'])
df.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Logarea,month_encoded,day_encoded
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292,0.482512,5.758221,2.736944
std,2.313778,1.229900,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818,0.607333,4.373275,1.925061
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,4.000000,4.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000,0.181844,6.000000,3.000000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000,0.879096,11.000000,4.000000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000,3.038159,11.000000,6.000000


#### OBSERVAÇÃO - 517 ELEMENTOS - FAREMOS O SAMPLING DE 50 ELEMENTOS

### 2- Geração de Amostras

#### 2.1- Geração de Amostras - Amostra Aleatoria Simples - COM reposição

In [11]:
df_aas_COM = df.sample(n=50, replace=False)
#    frac=0.05, 
#    replace=False, 
#    weights=None, 
#    random_state=None, 
#    axis=None, 
#    ignore_index=False)
df_aas_COM.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Logarea,month_encoded,day_encoded
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.0,50.000000,50.000000,50.000000,50.000000
mean,5.020000,4.480000,91.122000,109.374000,524.198000,9.534000,18.498000,41.440000,3.842000,0.0,19.163000,0.460463,5.800000,2.920000
std,2.180526,1.265556,3.796899,70.443377,267.940598,7.850533,6.342068,14.925885,1.810073,0.0,105.236809,0.591583,4.412413,1.816478
min,1.000000,2.000000,79.500000,3.000000,15.300000,1.100000,4.600000,17.000000,0.900000,0.0,0.000000,0.000000,0.000000,0.000000
25%,4.000000,4.000000,90.100000,48.975000,351.075000,6.400000,14.750000,29.000000,2.700000,0.0,0.000000,0.000000,1.000000,1.000000
50%,5.500000,4.000000,92.100000,102.300000,654.100000,8.400000,18.350000,40.000000,4.000000,0.0,1.040000,0.309583,5.500000,2.500000
75%,7.000000,5.000000,93.550000,137.000000,698.600000,9.825000,22.575000,47.750000,4.900000,0.0,4.385000,0.730695,11.000000,4.000000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,32.600000,80.000000,8.500000,0.0,746.280000,2.873483,11.000000,6.000000


#### 2.2- Geração de Amostras - Amostra Aleatoria Simples - SEM reposição

In [5]:
df_aas_SEM = df.sample(n=50, replace=True)
#    frac=0.05, 
#    replace=False, 
#    weights=None, 
#    random_state=None, 
#    axis=None, 
#    ignore_index=False)
df_aas_SEM.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Logarea,month_encoded,day_encoded
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000,50.0,50.000000,50.000000,50.000000,50.000000
mean,4.720000,4.540000,89.910000,108.816000,562.286000,8.08400,17.878000,46.360000,4.286000,0.0,6.073000,0.452490,5.400000,2.780000
std,2.339021,1.248836,6.784323,60.721318,218.330679,3.84354,7.197173,17.212667,1.859999,0.0,11.124889,0.563945,4.407785,1.961778
min,1.000000,2.000000,50.400000,3.700000,9.300000,0.40000,2.200000,21.000000,1.300000,0.0,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,89.025000,77.650000,399.200000,5.37500,14.325000,30.500000,2.700000,0.0,0.000000,0.000000,1.000000,1.000000
50%,4.000000,4.000000,91.500000,99.300000,665.600000,8.40000,18.750000,46.500000,4.000000,0.0,0.000000,0.000000,5.000000,3.000000
75%,6.750000,5.000000,93.225000,138.800000,714.800000,9.82500,21.275000,62.500000,5.400000,0.0,7.237500,0.915744,11.000000,4.000000
max,9.000000,9.000000,96.100000,231.100000,817.500000,20.00000,33.300000,79.000000,8.500000,0.0,49.590000,1.704065,11.000000,6.000000


#### 2.3 - Geração de Amostras - Amostragem Sistemática (AS)

In [6]:
# Criação de função que gera uma amostragem alaeatória sistemática
def systematic_sampling(df, step):
    selected_index = np.arange(1,len(df),step)
    # using iloc for getting thee data with selected indices
    systematic_sampling = df.iloc[selected_index]
    return(systematic_sampling)
N = 517
n = 50
step = int(N/n)
AS = systematic_sampling(df, step)
AS.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Logarea,month_encoded,day_encoded
count,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000,52.000000
mean,5.038462,4.442308,91.061538,106.551923,552.228846,9.290385,18.821154,43.307692,4.178846,0.015385,7.675577,0.524546,5.192308,2.480769
std,2.512413,1.304795,3.169657,55.199789,229.278931,4.295290,6.112999,17.948225,1.778889,0.110940,15.605608,0.576414,4.365814,2.004802
min,1.000000,2.000000,81.600000,3.200000,43.600000,1.900000,5.100000,18.000000,0.900000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,90.175000,72.050000,518.700000,7.025000,16.075000,29.000000,3.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,4.500000,4.000000,91.650000,102.050000,631.200000,8.700000,18.700000,39.500000,4.000000,0.000000,1.295000,0.360608,5.000000,2.000000
75%,7.250000,5.000000,92.875000,142.250000,710.675000,10.400000,22.925000,53.000000,4.900000,0.000000,8.170000,0.962151,11.000000,4.000000
max,9.000000,9.000000,96.100000,231.100000,795.300000,22.600000,32.400000,96.000000,9.400000,0.800000,95.180000,1.983085,11.000000,6.000000


#### 2.4 - Geração de Amostras - Amostragem por Conglomerados ou Clusters (AC)

In [7]:
# criação de função de amostragem por clusterização
def get_clustered_Sample(df, n_per_cluster, num_select_clusters):
    N = len(df)
    K = int(N/n_per_cluster)
    data = None
    for k in range(K):
        sample_k = df.sample(n_per_cluster)
        sample_k["cluster"] = np.repeat(k,len(sample_k))
        df = df.drop(index = sample_k.index)
        data = pd.concat([data,sample_k],axis = 0)

    random_chosen_clusters = np.random.randint(0,K,size = num_select_clusters)
    samples = data[data.cluster.isin(random_chosen_clusters)]
    return(samples)

AC = get_clustered_Sample(df = df, n_per_cluster = 10, num_select_clusters =5)
AC.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Logarea,month_encoded,day_encoded,cluster
count,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000
mean,5.225000,4.475000,91.172500,102.922500,518.900000,8.807500,18.510000,43.700000,4.340000,0.170000,4.984000,0.413364,5.650000,2.900000,23.000000
std,2.154155,1.012423,3.143164,67.538826,249.110407,4.017359,6.379768,14.834471,1.700648,1.012284,9.869387,0.519610,3.958438,1.836943,14.014644
min,1.000000,2.000000,81.600000,6.600000,18.700000,1.900000,2.200000,21.000000,1.800000,0.000000,0.000000,0.000000,1.000000,0.000000,4.000000
25%,3.750000,4.000000,90.550000,48.500000,351.875000,6.200000,15.025000,32.250000,2.700000,0.000000,0.000000,0.000000,1.000000,1.000000,16.750000
50%,5.500000,4.500000,91.600000,104.000000,628.000000,8.550000,18.650000,44.000000,4.250000,0.000000,0.340000,0.112655,5.500000,3.000000,22.500000
75%,7.000000,5.000000,92.925000,134.200000,696.100000,11.025000,22.500000,50.750000,5.025000,0.000000,6.665000,0.884479,10.000000,4.000000,28.750000
max,9.000000,6.000000,96.100000,273.800000,822.800000,17.900000,31.000000,78.000000,8.500000,6.400000,49.370000,1.702172,11.000000,6.000000,43.000000


#### 2.5 - Geração de Amostras - Amostragem Ponderada (AP)

In [8]:
def get_weighted_sample(df,n):
    def get_class_prob(x):
        weight_x = int(np.rint(n * len(x[x.Logarea != 0]) / len(df[df.Logarea != 0])))
        sampled_x = x.sample(weight_x).reset_index(drop=True)
        return (sampled_x)
        # we are grouping by the target class we use for the proportions

    weighted_sample = df.groupby('rain').apply(get_class_prob)
    print(weighted_sample["rain"].value_counts())
    return (weighted_sample)
AP = get_weighted_sample(df,50)
#sample
AP.describe()

0.0    50
Name: rain, dtype: int64


,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Logarea,month_encoded,day_encoded
count,50.000000,50.00000,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000,50.0,50.000000,50.000000,50.000000,50.000000
mean,5.080000,4.28000,89.184000,95.618000,525.03200,9.672000,18.220000,47.480000,4.274000,0.0,27.240800,0.462163,6.380000,2.260000
std,2.038807,1.16128,11.303248,63.073924,278.24036,7.991706,6.774141,19.346718,2.098669,0.0,153.859569,0.662634,4.384062,1.770622
min,1.000000,2.00000,18.700000,1.100000,7.90000,0.000000,4.600000,21.000000,0.900000,0.0,0.000000,0.000000,0.000000,0.000000
25%,3.250000,4.00000,90.150000,50.375000,310.70000,6.300000,14.075000,31.500000,2.700000,0.0,0.000000,0.000000,1.000000,0.250000
50%,5.500000,4.00000,91.650000,90.000000,680.45000,8.000000,19.100000,45.000000,4.250000,0.0,0.000000,0.000000,7.000000,2.000000
75%,6.750000,5.00000,93.200000,133.400000,723.57500,10.125000,22.875000,57.750000,5.400000,0.0,7.830000,0.945224,11.000000,4.000000
max,9.000000,9.00000,96.100000,266.200000,803.30000,56.100000,32.300000,100.000000,9.400000,0.0,1090.840000,3.038159,11.000000,6.000000


#### 2.6 - Geração de Amostras - Amostragem Estratificada (AE)

In [9]:
#### Adição de dimensão
price_vb  = pd.Series(np.random.uniform(1,4,size = N))
cat = pd.Series(np.random.choice([0,1],size = len(price_vb)))
df ['Categoria'] = cat
#### AE
def get_startified_sample(df,n,num_clusters_needed):
    N = len(df)
    num_obs_per_cluster = int(N/n)
    K = int(N/num_obs_per_cluster)

    def get_weighted_sample(df,num_obs_per_cluster):
        def get_sample_per_class(x):
            n_x = int(np.rint(num_obs_per_cluster*len(x[x.Categoria !=0])/len(df[df.Categoria !=0])))
            sample_x = x.sample(n_x)
            return(sample_x)
        weighted_sample = df.groupby("rain").apply(get_sample_per_class)
        return(weighted_sample)

    stratas = None
    for k in range(K):
        weighted_sample_k = get_weighted_sample(df,num_obs_per_cluster).reset_index(drop = True)
        weighted_sample_k["cluster"] = np.repeat(k,len(weighted_sample_k))
        stratas = pd.concat([stratas, weighted_sample_k],axis = 0)
        df.drop(index = weighted_sample_k.index)
    selected_strata_clusters = np.random.randint(0,K,size = num_clusters_needed)
    stratified_samples = stratas[stratas.cluster.isin(selected_strata_clusters)]
    return(stratified_samples)

AE = get_startified_sample(df = df,n = 100,num_clusters_needed = 10)
AE.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,Logarea,month_encoded,day_encoded,Categoria,cluster
count,50.000000,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.0,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,4.880000,4.400000,90.864000,97.86000,520.802000,8.592000,17.452000,41.780000,4.392000,0.0,4.925000,0.342576,6.820000,2.840000,0.540000,35.200000
std,2.282498,1.309307,2.819955,57.54124,262.848648,3.297546,5.819259,17.868328,1.938318,0.0,13.805034,0.498662,4.217021,1.972877,0.503457,26.625904
min,1.000000,2.000000,81.500000,9.10000,28.300000,2.100000,4.600000,15.000000,0.900000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3.000000,4.000000,90.125000,48.60000,322.475000,7.000000,14.450000,27.000000,3.100000,0.0,0.000000,0.000000,2.000000,1.000000,0.000000,18.000000
50%,5.000000,4.000000,91.750000,102.30000,652.650000,8.650000,18.300000,37.000000,4.000000,0.0,0.000000,0.000000,7.000000,3.000000,1.000000,31.500000
75%,7.000000,5.000000,92.575000,126.50000,723.575000,9.600000,20.625000,53.750000,5.700000,0.0,2.385000,0.528287,11.000000,4.000000,1.000000,49.000000
max,9.000000,9.000000,94.800000,287.20000,849.300000,17.900000,29.600000,87.000000,9.400000,0.0,86.450000,1.941760,11.000000,6.000000,1.000000,99.000000


In [10]:
EstDescritiva = pd.DataFrame()
index1 = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
EstDescritiva1 = pd.DataFrame(index=index1)
EstDescritiva1 ['Original'] = (df.area.describe())
EstDescritiva1  ['AAS-COM'] = (df_aas_COM.area.describe())
EstDescritiva1  ['AAS-SEM'] = (df_aas_SEM.area.describe())
EstDescritiva1  ['Sistemática'] = (AS.area.describe())
EstDescritiva1  ['Ponderada'] = (AP.area.describe())
EstDescritiva1  ['Cluster'] = (AC.area.describe())
EstDescritiva1  ['AE'] = (AE.area.describe())
EstDescritiva1

,Original,AAS-COM,AAS-SEM,Sistemática,Ponderada,Cluster,AE
count,517.000000,50.00000,50.000000,52.000000,50.000000,40.000000,50.000000
mean,12.847292,9.81860,6.073000,7.675577,27.240800,4.984000,4.925000
std,63.655818,29.00184,11.124889,15.605608,153.859569,9.869387,13.805034
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.520000,0.92500,0.000000,1.295000,0.000000,0.340000,0.000000
75%,6.570000,6.76500,7.237500,8.170000,7.830000,6.665000,2.385000
max,1090.840000,174.63000,49.590000,95.180000,1090.840000,49.370000,86.450000
